In [ ]:
import clickhouse_connect

client = clickhouse_connect.get_client(host='localhost', port=8123,username='text2sql', password='password')

In [ ]:
def get_clickhouse_table_schema():
    schema = ""
    result = client.query("SHOW tables")
    for table in result.result_rows:
        table_name = table[0]
        create_sql = (
            client.query("SHOW CREATE {0}".format(table_name)).result_rows[0][0] + ";"
        )
        schema += create_sql+"\n"
    return schema


schema = get_clickhouse_table_schema()
print(schema)

CREATE TABLE default.dish
(
    `id` UInt32,
    `name` String,
    `description` String,
    `menus_appeared` UInt32,
    `times_appeared` Int32,
    `first_appeared` UInt16,
    `last_appeared` UInt16,
    `lowest_price` Decimal(18, 3),
    `highest_price` Decimal(18, 3)
)
ENGINE = MergeTree
ORDER BY id
SETTINGS index_granularity = 8192;
CREATE TABLE default.menu
(
    `id` UInt32,
    `name` String,
    `sponsor` String,
    `event` String,
    `venue` String,
    `place` String,
    `physical_description` String,
    `occasion` String,
    `notes` String,
    `call_number` String,
    `keywords` String,
    `language` String,
    `date` String,
    `location` String,
    `location_type` String,
    `currency` String,
    `currency_symbol` String,
    `status` String,
    `page_count` UInt16,
    `dish_count` UInt16
)
ENGINE = MergeTree
ORDER BY id
SETTINGS index_granularity = 8192;
CREATE TABLE default.menu_item
(
    `id` UInt32,
    `menu_page_id` UInt32,
    `price` Decimal(18, 

In [24]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import sqlite3
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
tokenizer = AutoTokenizer.from_pretrained(
    "../deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    "../deepseek-ai/deepseek-coder-6.7b-instruct",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
).cuda()

/date1/luzhan/environment/miniconda3/envs/deepseek/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.75s/it]


In [25]:
import re

pattern = re.compile(r"```sql (.+) ```", re.S)

system_prompt = "### Instruction:\nGiven the following ClickHouse database schema:\n"


def text2sql(
    schema: str,
    question: str,
):
    # conversation = [
    #     {"role": "system", "content": system_prompt+schema},
    #     # {"role": "user", "content": "### Complete sqlite SQL query only and with no explanation \n ### Sqlite SQL tables, with their properties: \n# \n# stadium(Stadium_ID,Location,Name,Capacity,Highest,Lowest,Average)\n;# singer(Singer_ID,Name,Country,Song_Name,Song_release_year,Age,Is_male)\n;# concert(concert_ID,concert_Name,Theme,Stadium_ID,Year)\n;# singer_in_concert(concert_ID,Singer_ID)\n.# \n ### How many singers do we have? \n SELECT"},
    #     # {"role": "system", "content": schema},
    #     {"role": "user", "content": question+"\nSELECT"},
    # ]
    conversation = (
        system_prompt
        + schema
        + "Write the ClickHouse SQL query to answer this question: "
        + question
        + "\n### Response:\n"
    )

    input_text = tokenizer(conversation, return_tensors="pt").to(model.device)
    # input_text = tokenizer.apply_chat_template(conversation, return_tensors="pt").to(model.device)  # type: ignore

    outputs = model.generate(
        **input_text,
        # temperature=1.0,
        # do_sample=True,
        max_new_tokens=512,
        # repetition_penalty=1.5,
        pad_token_id=0,
        bos_token_id=32013,
        eos_token_id=32021,
    )
    # [len(input_text[0]) :]
    answer = tokenizer.decode(
        outputs[0], skip_special_tokens=True
    )
    return answer


question="Which tables are in the database"

answer = text2sql(get_clickhouse_table_schema(), question)
print(answer)

### Instruction:
Given the following ClickHouse database schema:
CREATE TABLE default.dish
(
    `id` UInt32,
    `name` String,
    `description` String,
    `menus_appeared` UInt32,
    `times_appeared` Int32,
    `first_appeared` UInt16,
    `last_appeared` UInt16,
    `lowest_price` Decimal(18, 3),
    `highest_price` Decimal(18, 3)
)
ENGINE = MergeTree
ORDER BY id
SETTINGS index_granularity = 8192;
CREATE TABLE default.menu
(
    `id` UInt32,
    `name` String,
    `sponsor` String,
    `event` String,
    `venue` String,
    `place` String,
    `physical_description` String,
    `occasion` String,
    `notes` String,
    `call_number` String,
    `keywords` String,
    `language` String,
    `date` String,
    `location` String,
    `location_type` String,
    `currency` String,
    `currency_symbol` String,
    `status` String,
    `page_count` UInt16,
    `dish_count` UInt16
)
ENGINE = MergeTree
ORDER BY id
SETTINGS index_granularity = 8192;
CREATE TABLE default.menu_item
(
   